#### Encoding Geography
# Community Health Assessment Tool Backend
Developed by Jessica Embury, San Diego State University, Department of Geography, 2023

** Go to Runtime >> Run All before using the web app:
https://encgeo-chat.anvil.app/

#### Notebook Set Up & Variable Definition

In [1]:
!pip install anvil-uplink
!pip install scikit-criteria==0.2.11
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45228 sha256=d95f82a849a77f46024e7f7ecc37343793993864330b63f2176edb3ca575dc86
  Stored in directory: /root/.cache/pip/wheels/2e/7c/ad/d9c746276bf024d44296340869fcb169f1e5d80fb147351a57
Successfully built ws4py


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 52.9 MB/s eta 0:00:00
  Created wheel for scikit-criteria: filename=scikit_criteria-0.2.11-py3-none-any.whl size=116392 sha256=788288e91b250064f2bd649a4eeed420cfbbf1acb9765fbe004dfc171cd86c9b
  Stored in directory: /root/.cache/pip/wheels/ee/9c/cd/c584a0f52e8046a090651d3183635ba4e85ed548f1ecd43db3
Successfully built scikit-criteria
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 46.9 MB/s eta 0:00:00


In [2]:
import anvil.server

In [3]:
anvil.server.connect("server_SZNFS3LZEGXNK7PNS2FN7NOC-QLCKK24NWE7Z3W5Z")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [4]:
import geopandas as gpd
import json
import math
import numpy as np
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import plotly.express as px
import plotly.graph_objects as go
import requests
from skcriteria.madm import simple # scikit-criteria==0.2.11
from skcriteria import Data as skData


In [5]:
health_dict = {'Percent of Residents With Cancer':'cancer_pct', 'Percent of Residents With Chronic Obstructive Pulmonary Disease (COPD)': 'copd_pct', 'Percent of Residents With Coronary Heart Disease':'chd_pct', 'Percent of Residents With Diabetes':'diabetes_pct', 'Percent of Residents With High Blood Pressure':'hbp_pct', 'Percent of Residents With Obesity (BMI > 30)':'bmi30plus_pct',  'Percent of Residents With Stroke':'stroke_pct'}
sociodem_dict = {'Median Annual Household Income':'median_income', 'Percent of Households Receiving Food or Cash Assistance':'hhs_snap_pct', 'Percent of Workers With Active Commutes':'active_commute', 'Percent of Residents With Auto Access':'auto_access', 'Percent of Residents With Health Insurance':'insurance', 'Percent of Residents With Low Physical Activity':'lowphysactiv_pct'}                
environ_dict = {'Number of Fast Food Restaurants':'num_fast_food', 'Number of Full-Service Restaurants':'num_full_restaurants', 'Number of Snack and Nonalcoholic Beverage Bars':'num_snack_nonalcbev_bars', 'Percent of Residents With Nearby Park Access':'park_access', 'Percent of Residents With Nearby Public Transit Access':'transit_access', 'Percent of Residents With Nearby Supermarket Access':'supermkt_access', 'Neighborhood Walkability Score':'walkability'}
var_min_max = {'bmi30plus_pct':'min', 'cancer_pct':'min', 'copd_pct':'min', 'chd_pct':'min', 'diabetes_pct':'min', 'hbp_pct':'min', 'stroke_pct':'min', 'hhs_snap_pct':'min', 'active_commute':'max', 'auto_access':'max', 'insurance':'max', 'median_income':'max', 'lowphysactiv_pct':'min', 'park_access':'max', 'supermkt_access':'max', 'transit_access':'max', 'walkability':'max', 'num_full_restaurants':'min', 'num_fast_food':'min', 'num_snack_nonalcbev_bars':'min'}
color_map = {'0':'#e8e8e8', '1':'#ace4e4', '2':'#5ac8c8', '3':'#dfb0d6', '4':'#a5add3', '5':'#5698b9', '6':'#be64ac', '7':'#8c62aa', '8':'#3b4994'}
asset_categories = {'Convenience Stores':'Convenience Store', 'Farmers Markets':'Farmer\'s Market', 'Produce Specialty Stores':'Fruits/Veg Specialty', 'Grocery Stores':'Grocery Store', 'Health Clinics':'Clinic', 'Hospitals':'Hospital', 'Parks':'Park', 'Supermarkets':'Supermarket'}
school_url = 'https://github.com/jlembury/community_health_assessment/blob/main/xwalk_school_sra.csv?raw=true'  
tract_url = 'https://github.com/jlembury/community_health_assessment/blob/main/data_tracts_risk.csv?raw=true' 
geo_school_url = 'https://github.com/jlembury/community_health_assessment/blob/main/xwalk_school_sra.geojson?raw=true'
geo_sra_url = 'https://github.com/jlembury/community_health_assessment/blob/main/sra_boundaries.geojson?raw=true'
geo_tract_url = 'https://github.com/jlembury/community_health_assessment/blob/main/data_tracts_risk.geojson?raw=true'
assets_url = 'https://github.com/jlembury/community_health_assessment/blob/main/data_assets.csv?raw=true'

#### Functions for Part 1. Health Mapping

In [12]:
# bivar mapping support functions
def conf_defaults():
    # Define some variables for later use
    conf = {
        'plot_title': 'Bivariate choropleth map using Ploty',  # Title text
        'plot_title_size':18,
        'width': 1000,  # Width of the final map container
        'ratio': 0.8,  # Ratio of height to width
        'center_lat': 0,  # Latitude of the center of the map
        'center_lon': 0,  # Longitude of the center of the map
        'map_zoom': 3,  # Zoom factor of the map
        'hover_x_label': 'Label x variable',  # Label to appear on hover
        'hover_y_label': 'Label y variable',  # Label to appear on hover
        'borders_width': 0.5,  # Width of the geographic entity borders
        'borders_color': '#f8f8f8',  # Color of the geographic entity borders

        # Define settings for the legend
        'top': 1,  # Vertical position of the top right corner (0: bottom, 1: top)
        'right': 1,  # Horizontal position of the top right corner (0: left, 1: right)
        'box_w': 0.06,  # Width of each rectangle
        'box_h': 0.06,  # Height of each rectangle
        'line_color': '#f8f8f8',  # Color of the rectagles' borders
        'line_width': 0,  # Width of the rectagles' borders
        'legend_x_label': 'Higher x value',  # x variable label for the legend 
        'legend_y_label': 'Higher y value',  # y variable label for the legend
        'legend_font_size': 9,  # Legend font size
        'legend_font_color': '#333',  # Legend font color
    }

    # Calculate height
    conf['height']= conf['width'] * conf['ratio']
    
    return conf

"""
Function to recalculate values in case width is changed
"""
def recalc_vars(new_width, variables, conf=conf_defaults()):
    
    # Calculate the factor of the changed width
    factor = new_width / 1000
    
    # Apply factor to all variables that have been passed to the function
    for var in variables:
        if var == 'map_zoom':
            # Calculate the zoom factor
            # Mapbox zoom is based on a log scale. map_zoom needs to be set 
            # to value ideal for our map at 1000px.
            # So factor = 2 ^ (zoom - map_zoom) and zoom = log(factor) / log(2) + map_zoom
            conf[var] = math.log(factor) / math.log(2) + conf[var]
        else:
            conf[var] = conf[var] * factor

    return conf

def set_interval_value(x, break_1, break_2):
    if x <= break_1: 
        return 0
    elif break_1 < x <= break_2: 
        return 1
    else: 
        return 2

def prepare_df(df, x='x', y='y'):
    
    # Check if arguments match all requirements
    if df[x].shape[0] != df[y].shape[0]:
        raise ValueError('ERROR: The list of x and y coordinates must have the same length.')
    
    # Calculate break points at percentiles 33 and 66
    x_breaks = np.percentile(df[x], [33, 66])
    y_breaks = np.percentile(df[y], [33, 66])
    
    # Assign values of both variables to one of three bins (0, 1, 2)
    x_bins = [set_interval_value(value_x, x_breaks[0], x_breaks[1]) for value_x in df[x]]
    y_bins = [set_interval_value(value_y, y_breaks[0], y_breaks[1]) for value_y in df[y]]
    
    # Calculate the position of each x/y value pair in the 9-color matrix of bivariate colors
    df['biv_bins'] = [int(value_x + 3 * value_y) for value_x, value_y in zip(x_bins, y_bins)]
    
    return df

def create_legend(fig, colors, conf=conf_defaults()):
    
    # Reverse the order of colors
    legend_colors = [color_map[key] for key in color_map]
    legend_colors.reverse()

    # Calculate coordinates for all nine rectangles
    coord = []

    # Adapt height to ratio to get squares
    width = conf['box_w']
    height = conf['box_h']/conf['ratio']
    
    # Start looping through rows and columns to calculate corners the squares
    for row in range(1, 4):
        for col in range(1, 4):
            coord.append({
                'x0': round(conf['right']-(col-1)*width, 4),
                'y0': round(conf['top']-(row-1)*height, 4),
                'x1': round(conf['right']-col*width, 4),
                'y1': round(conf['top']-row*height, 4)
            })

    # Create shapes (rectangles)
    for i, value in enumerate(coord):
        # Add rectangle
        fig.add_shape(go.layout.Shape(
            type='rect',
            fillcolor=legend_colors[i],
            line=dict(
                color=conf['line_color'],
                width=conf['line_width'],
            ),
            xref='paper',
            yref='paper',
            xanchor='right',
            yanchor='top',
            x0=coord[i]['x0'],
            y0=coord[i]['y0'],
            x1=coord[i]['x1'],
            y1=coord[i]['y1'],
        ))
    
        # Add text for first variable
        fig.add_annotation(
            xref='paper',
            yref='paper',
            xanchor='left',
            yanchor='top',
            x=coord[8]['x1'],
            y=coord[8]['y1'],
            showarrow=False,
            text=conf['legend_x_label'] + ' 🠒',
            font=dict(
                color=conf['legend_font_color'],
                size=conf['legend_font_size'],
            ),
            borderpad=0,
        )
        
        # Add text for second variable
        fig.add_annotation(
            xref='paper',
            yref='paper',
            xanchor='right',
            yanchor='bottom',
            x=coord[8]['x1'],
            y=coord[8]['y1'],
            showarrow=False,
            text=conf['legend_y_label'] + ' 🠒',
            font=dict(
                color=conf['legend_font_color'],
                size=conf['legend_font_size'],
            ),
            textangle=270,
            borderpad=0,
            )
    
    return fig

In [14]:
@anvil.server.callable
def plot_bivar_choro(hm_school_dd, hm_var1, hm_var2):
  health_var = health_dict[hm_var1]
  try:
    var2 = sociodem_dict[hm_var2]
  except:
    var2 = environ_dict[hm_var2]
  
  school_gdf = gpd.read_file(geo_school_url).query('name == @hm_school_dd')[['name', 'sra', 'geometry']]
  school_geom = school_gdf['geometry'].iloc[0]
  school_df = pd.DataFrame(columns=['name', 'x', 'y'], data=[[hm_school_dd, school_geom.x, school_geom.y]])
  sra = school_gdf['sra'].iloc[0]
  sra_gdf = gpd.read_file(geo_sra_url).query('sra == @sra')

  tracts_gdf = gpd.read_file(geo_tract_url).query('sra == @sra')[['tract', 'sra', health_var, var2, 'geometry']]
  tracts_gdf = tracts_gdf.rename(columns={'tract':'Tract', 'sra':'SRA', health_var:hm_var1, var2:hm_var2})
  tracts_gdf[hm_var1] = (tracts_gdf[hm_var1].round(2)*100).astype(int)
  if 'Percent' in hm_var2:
    tracts_gdf[hm_var2] = (tracts_gdf[hm_var2].round(2)*100).astype(int)
  else:
    tracts_gdf[hm_var2] = (tracts_gdf[hm_var2].round(0)).astype(int)
  
  # map config settings
  conf = conf_defaults()
  conf['plot_title'] = 'Bivariate Map of {} <br>& {} in {}'.format(hm_var1, hm_var2, sra.title())
  conf['hover_x_label'] = '{}'.format(hm_var2)  # Label to appear on hover
  conf['hover_y_label'] = '{}'.format(hm_var1)  # Label to appear on hover
  conf['width'] = 1000
  conf['center_lat'] = sra_gdf['geometry'].iloc[0].centroid.y
  conf['center_lon'] = sra_gdf['geometry'].iloc[0].centroid.x  # Longitude of the center of the map
  conf['map_zoom'] = 11  # Zoom factor of the map
  conf['borders_width'] = 0  # Width of the geographic entity borders

  # Define settings for the legend
  conf['top'] = 0.225  # Vertical position of the top right corner (0: bottom, 1: top)
  conf['right'] = 0.18  # Horizontal position of the top right corner (0: left, 1: right)
  conf['line_width'] = 0  # Width of the rectagles' borders
  conf['legend_x_label'] = 'Higher {}'.format(hm_var2)   # x variable label for the legend 
  conf['legend_y_label'] = 'Higher {}'.format(hm_var1)  # y variable label for the legend

  # Recalculate values if width differs from default
  if not conf['width'] == 1000:             
      conf = recalc_vars(conf['width'], ['height', 'plot_title_size', 'legend_font_size', 'map_zoom'], conf)
        
  # Prepare the dataframe with the necessary information for our bivariate map
  df_plot = prepare_df(tracts_gdf, hm_var2, hm_var1)
  df_plot['biv_bins'] = df_plot['biv_bins'].astype(str)
  fig = px.choropleth_mapbox(tracts_gdf,
                   geojson=tracts_gdf.geometry,
                   locations=tracts_gdf.index,
                   color='biv_bins', mapbox_style = 'open-street-map', color_discrete_map=color_map,
                   hover_data={hm_var1:True, hm_var2:True},
                   hover_name = tracts_gdf.Tract, opacity=0.5) 

  # Add some more details
  fig.update_layout(title=dict(text=conf['plot_title'],),
        width=conf['width'],
        height=conf['height'],
        autosize=True,
        showlegend=False,
        mapbox=dict(
            center=dict(lat=conf['center_lat'], lon=conf['center_lon']),  # Set map center
            zoom=conf['map_zoom'],  # Set zoom
        ))

  fig.update_traces(
        marker_line_width=conf['borders_width'],  # Width of the geographic entity borders
        marker_line_color=conf['borders_color'],  # Color of the geographic entity borders
        showscale=False,  # Hide the colorscale
  )

  fig2 = px.scatter_mapbox(school_df, lat='y', lon='x', color='name', hover_name='name', hover_data={'x':False, 'y':False})
  fig.add_trace(fig2.data[0])
  fig.update_layout(legend_orientation="h")

  # Add the legend
  fig = create_legend(fig, color_map, conf)
    
  return fig

#### Functions for Part 2. Community Health Assessment Tool

In [6]:
@anvil.server.callable
def get_schools(zipcode):
  school_df = pd.read_csv(school_url)
  schools_list = list(school_df.query('zip == @zipcode')['name'])
  schools_list.sort()
  return schools_list

In [7]:
@anvil.server.callable
def get_school_sra(schools_dropdown):
  school_df = pd.read_csv(school_url)
  sra = school_df.query('name == @schools_dropdown').reset_index()['sra'][0]
  sra_title = sra.title()
  tract_df = pd.read_csv(tract_url)
  tract_subset = tract_df.query('sra == @sra')
  num_tracts = len(tract_subset)
  text = f'{schools_dropdown} is in the {sra_title} Sub-Regional Area. \n {sra_title} has {num_tracts} Census Tracts.'
  return text

In [8]:
@anvil.server.callable
def run_mcda(schools_dropdown, health_var, health_weight, sociodem_var, sociodem_weight, environ_var, environ_weight):
  school_df = pd.read_csv(school_url)
  sra = school_df.query('name == @schools_dropdown').reset_index()['sra'][0]
  tract_df = pd.read_csv(tract_url).query('sra == @sra')
  
  data = tract_df[['tract', health_dict[health_var], sociodem_dict[sociodem_var], environ_dict[environ_var]]]
  data = data.replace(0, 0.000001)
  attribs = [health_dict[health_var], sociodem_dict[sociodem_var], environ_dict[environ_var]]
  tracts = list(data['tract'])
  data.drop(['tract'], axis=1, inplace=True)
  direction = [var_min_max[health_dict[health_var]], var_min_max[sociodem_dict[sociodem_var]], var_min_max[environ_dict[environ_var]]]
  weights_total = health_weight + sociodem_weight + environ_weight
  weights = [health_weight/weights_total, sociodem_weight/weights_total, environ_weight/weights_total]

  criteria_data = skData(data, direction, weights, tracts, attribs)
  dm = simple.WeightedSum(mnorm="sum")
  dec = dm.decide(criteria_data)
  tract_df['Score'] = dec.e_.points
  tract_df['Rank'] = dec.rank_
  df = tract_df.sort_values(by='Rank').reset_index().rename(columns={'tract':'Tract'})[['Tract', 'Rank', 'Score', health_dict[health_var], sociodem_dict[sociodem_var], environ_dict[environ_var]]]
  if 'Percent' in health_var:
    df[health_dict[health_var]] = (df[health_dict[health_var]].round(2)*100).astype(int)
  if 'Percent' in sociodem_var:
    df[sociodem_dict[sociodem_var]] = (df[sociodem_dict[sociodem_var]].round(2)*100).astype(int)
  if 'Percent' in environ_var:
    df[environ_dict[environ_var]] = (df[environ_dict[environ_var]].round(2)*100).astype(int)
  else:
    df[environ_dict[environ_var]] = (df[environ_dict[environ_var]].round(0)).astype(int)

  results = [{'Tract': df['Tract'].iloc[x], 'Rank':df['Rank'].iloc[x], health_var:df[health_dict[health_var]].iloc[x], sociodem_var:df[sociodem_dict[sociodem_var]].iloc[x], environ_var:df[environ_dict[environ_var]].iloc[x]} for x in df.index]
  return results

In [9]:
@anvil.server.callable
def plot_choro(schools_dropdown, results, health_weight, sociodem_weight, environ_weight):
  school_gdf = gpd.read_file(geo_school_url).query('name == @schools_dropdown')
  sra = school_gdf['sra'].iloc[0]
  school_geom = school_gdf['geometry'].iloc[0]
  school_df = pd.DataFrame(columns=['name', 'x', 'y'], data=[[schools_dropdown, school_geom.x, school_geom.y]])
  sra_gdf = gpd.read_file(geo_sra_url).query('sra == @sra')
  centroid = [sra_gdf['geometry'].iloc[0].centroid.y, sra_gdf['geometry'].iloc[0].centroid.x]

  tract_ids = [str(results[x]['Tract']) for x in range(len(results))]
  cols = [key for key in results[0]]
  cols_abbrev = ['tract', health_dict[cols[2]], sociodem_dict[cols[3]], environ_dict[cols[4]], 'geometry']
  tracts_gdf = gpd.read_file(geo_tract_url).query('tract == @tract_ids')
  tracts_gdf['rank'] = 0
  for i in range(len(results)):
    tracts_gdf.loc[tracts_gdf.tract == str(results[i]['Tract']), 'rank'] = results[i]['Rank']
  tracts_gdf = tracts_gdf.rename(columns={'tract':'Tract', 'rank':'Rank', health_dict[cols[2]]:cols[2], sociodem_dict[cols[3]]:cols[3], environ_dict[cols[4]]:cols[4]})[['Tract', 'Rank', cols[2], cols[3], cols[4], 'geometry']]
  if 'Percent' in cols[2]:
    tracts_gdf[cols[2]] = (tracts_gdf[cols[2]].round(2)*100).astype(int)
  if 'Percent' in cols[3]:
    tracts_gdf[cols[3]] = (tracts_gdf[cols[3]].round(2)*100).astype(int)
  if 'Percent' in cols[4]:
    tracts_gdf[cols[4]] = (tracts_gdf[cols[4]].round(2)*100).astype(int)
  else:
    tracts_gdf[cols[4]] = (tracts_gdf[cols[4]].round(0)).astype(int)
  tracts_gdf['tract'] = tracts_gdf['Tract']

  fig = px.choropleth_mapbox(tracts_gdf,
                   geojson=tracts_gdf.geometry,
                   locations=tracts_gdf.index,
                   color='Rank', mapbox_style = 'open-street-map', zoom = 11, center = dict(lat = centroid[0], lon = centroid[1]),
                   hover_data={'Rank':True, cols[2]:True, cols[3]:True, cols[4]:True},
                   color_continuous_scale='rdbu_r', hover_name = tracts_gdf.tract, opacity=0.5,
                   title='Community Health Assessment for {}<br><sub>Health Criterion: {} (Weight {})<br>Socio-Demographic Criterion: {} (Weight {})<br>Environmental Criterion: {} (Weight {})</sub>'.format(sra.title(), cols[2], health_weight, cols[3], sociodem_weight, cols[4], environ_weight))
  fig2 = px.scatter_mapbox(school_df, lat='y', lon='x', hover_name='name', hover_data={'x':False, 'y':False})
  fig.add_trace(fig2.data[0])
  #fig.update_geos(fitbounds='locations', visible=False)
  fig.update_layout(margin=dict(t=150, b=10, r=20, l=20),legend_orientation="h")

  return fig

In [10]:
@anvil.server.callable
def get_sra_geom(schools_dropdown):
  school_gdf = gpd.read_file(geo_school_url).query('name == @schools_dropdown')
  sra = school_gdf['sra'].iloc[0]
  sra_gdf = gpd.read_file(geo_sra_url).query('sra == @sra')
  lonlat = list(sra_gdf['geometry'].iloc[0].exterior.coords)
  latlon = [(x[1], x[0]) for x in lonlat]
  centroid = [sra_gdf['geometry'].iloc[0].centroid.y, sra_gdf['geometry'].iloc[0].centroid.x]
  sra_info = {'sra_name':sra_gdf['sra'].iloc[0].title(), 'sra_geometry':latlon, 'sra_centroid': centroid}
  return sra_info

In [11]:
@anvil.server.callable
def get_school_coords(schools_dropdown):
  gdf = gpd.read_file(geo_school_url).query('name == @schools_dropdown')
  school_geom = gdf['geometry'].iloc[0]
  school_latlon = [school_geom.y, school_geom.x]
  return school_latlon

#### Functions for Part 3. Asset Mapping

In [15]:
@anvil.server.callable
def mcda4assetmapping(pd1, pd2, pw1, pd3, pw2, pd4, pw3):
  sra = pd1.upper()
  tract_df = pd.read_csv(tract_url).query('sra == @sra')
  
  data = tract_df[['tract', health_dict[pd2], sociodem_dict[pd3], environ_dict[pd4]]]
  data = data.replace(0, 0.000001)
  attribs = [health_dict[pd2], sociodem_dict[pd3], environ_dict[pd4]]
  tracts = list(data['tract'])
  data.drop(['tract'], axis=1, inplace=True)
  direction = [var_min_max[health_dict[pd2]], var_min_max[sociodem_dict[pd3]], var_min_max[environ_dict[pd4]]]
  weights_total = pw1 + pw2 + pw3
  weights = [pw1/weights_total, pw2/weights_total, pw3/weights_total]

  criteria_data = skData(data, direction, weights, tracts, attribs)
  dm = simple.WeightedSum(mnorm="sum")
  dec = dm.decide(criteria_data)
  tract_df['Rank'] = dec.rank_
  df = tract_df.sort_values(by='Rank').reset_index().rename(columns={'tract':'Tract'})[['Tract', 'Rank', health_dict[pd2], sociodem_dict[pd3], environ_dict[pd4]]]
  df[health_dict[pd2]] = (df[health_dict[pd2]].round(2)*100).astype(int)
  if 'Percent' in pd3:
    df[sociodem_dict[pd3]] = (df[sociodem_dict[pd3]].round(2)*100).astype(int)
  if 'Percent' in pd4:
    df[environ_dict[pd4]] = (df[environ_dict[pd4]].round(2)*100).astype(int)
  elif 'Walkability' in pd4:
    df[environ_dict[pd4]] = df[environ_dict[pd4]].astype(int)

  results = [{'Tract': df['Tract'].iloc[x], 'Rank':df['Rank'].iloc[x], pd2:df[health_dict[pd2]].iloc[x], pd3:df[sociodem_dict[pd3]].iloc[x], pd4:df[environ_dict[pd4]].iloc[x]} for x in df.index]
  return results

In [16]:
@anvil.server.callable
def plot_asset_map(results, assets, pd1, pw1, pw2, pw3):
  sra = pd1.upper()
  sra_gdf = gpd.read_file(geo_sra_url).query('sra == @sra')
  centroid = [sra_gdf['geometry'].iloc[0].centroid.y, sra_gdf['geometry'].iloc[0].centroid.x]

  tract_ids = [str(results[x]['Tract']) for x in range(len(results))]
  cols = [key for key in results[0]]
  cols_abbrev = ['tract', health_dict[cols[2]], sociodem_dict[cols[3]], environ_dict[cols[4]], 'geometry']
  tracts_gdf = gpd.read_file(geo_tract_url).query('tract == @tract_ids')
  tracts_gdf['rank'] = 0
  for i in range(len(results)):
    tracts_gdf.loc[tracts_gdf.tract == str(results[i]['Tract']), 'rank'] = results[i]['Rank']
  tracts_gdf = tracts_gdf.rename(columns={'tract':'Tract', 'rank':'Rank', health_dict[cols[2]]:cols[2], sociodem_dict[cols[3]]:cols[3], environ_dict[cols[4]]:cols[4]})[['Tract', 'Rank', cols[2], cols[3], cols[4], 'geometry']]
  tracts_gdf[cols[2]] = (tracts_gdf[cols[2]].round(2)*100).astype(int)
  if 'Percent' in cols[3]:
    tracts_gdf[cols[3]] = (tracts_gdf[cols[3]].round(2)*100).astype(int)
  if 'Percent' in cols[4]:
    tracts_gdf[cols[4]] = (tracts_gdf[cols[4]].round(2)*100).astype(int)
  elif 'Walkability' in cols[4]:
    tracts_gdf[cols[4]] = tracts_gdf[cols[4]].astype(int)
  tracts_gdf['tract'] = tracts_gdf['Tract']

  asset_cats = [asset_categories[x] for x in assets]
  assets_df = pd.read_csv(assets_url).query('sra == @sra').query('category == @asset_cats')

  fig = px.choropleth_mapbox(tracts_gdf,
                   geojson=tracts_gdf.geometry,
                   locations=tracts_gdf.index,
                   color='Rank', mapbox_style = 'open-street-map', zoom = 11, center = dict(lat = centroid[0], lon = centroid[1]),
                   hover_data={'Rank':True, cols[2]:True, cols[3]:True, cols[4]:True},
                   color_continuous_scale='rdbu_r', hover_name = tracts_gdf.tract, opacity=0.5,
                   title='Community Health & Asset Map for {}<br><sub>Health Criterion: {} (Weight {})<br>Socio-Demographic Criterion: {} (Weight {})<br>Environmental Criterion: {} (Weight {})</sub>'.format(pd1, cols[2], pw1, cols[3], pw2, cols[4], pw3))
  fig2 = px.scatter_mapbox(assets_df, lat='lat', lon='lon', color='category', hover_name='name', hover_data={'category':True, 'sra':False, 'lat':False, 'lon':False})
  for i in range(len(asset_cats)):
    fig.add_trace(fig2.data[i-1])
  fig.update_layout(margin=dict(t=150, b=10, r=20, l=20),legend_orientation="h")

  return fig

#### Keep Server Active for Web App

In [ ]:
anvil.server.wait_forever()